## Reinforcement Learning Models

### Imports and Utils

In [38]:
"""
Importing necessary libraries
"""
import pygame
import neat
import os
import random
import numpy as np

import pickle

In [26]:
"""
Defining Utilities for the Game
"""
WIN_WIDTH = 1920
WIN_HEIGHT = 1080
CAR_WIDTH = 60
CAR_HEIGHT = 60

CAR_IMG = pygame.transform.scale(pygame.image.load(os.path.join("Images", "car.png")), (CAR_WIDTH, CAR_HEIGHT))
ROAD_IMG = pygame.transform.scale(pygame.image.load(os.path.join("Images", "map1.png")), (WIN_WIDTH, WIN_HEIGHT))
BORDER_COLOR = (255, 255, 255, 255)

In [27]:
"""
Defining Car Class
"""
class Car:
    IMGS = CAR_IMG
    ROT_VEL = 5

    def __init__(self, position):
        self.position = position
        self.tilt = 0
        self.vel = 0

        self.image = self.IMGS
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
        self.radars = []

    def engine(self, action):
        if action[0] == 1:
            self.vel += 2
            self.vel = min(self.vel, 10)
        if action[1] == 1:
            self.vel -= 2
            self.vel = max(self.vel, 0)
        if action[2] == 1:
            self.tilt += self.ROT_VEL
            self.tilt = self.tilt % 360
        if action[3] == 1:
            self.tilt -= self.ROT_VEL
            self.tilt = self.tilt % 360

    def move(self):
        self.position[0] += self.vel*np.cos(self.tilt)
        self.position[1] += self.vel*np.sin(self.tilt)
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
    
    def draw(self, window, DRAW_LINES = False):
        rotated_image = pygame.transform.rotate(self.image, self.tilt)
        new_rect = rotated_image.get_rect(center=self.image.get_rect(topleft=(self.x, self.y)).center)
        window.blit(rotated_image, new_rect.topleft)

        if DRAW_LINES:
            for radar in self.radars:
                pygame.draw.line(window, (255, 0, 0), self.center, radar, 1)
                pygame.draw.circle(window, (255, 0, 0), radar, 5)

    def collide(self, map):
        car_mask = pygame.mask.from_threshold(self.image, (255, 255, 255), (1, 1, 1, 255))
        map_mask = pygame.mask.from_threshold(map.image, (255, 255, 255), (0, 0, 0, 255))

        offset = (int(self.position[0] - map.position[0]), int(self.position[1] - map.position[1]))
        point = car_mask.overlap(map_mask, offset)
        return point is not None

In [28]:
"""
Defining Map Class
"""

class Map:

    def __init__(self, position):
        self.position = position
        self.image = ROAD_IMG

    def draw(self, window):
        window.blit(self.image, self.position)        

In [30]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 150
RESTART_Y = 300

class RacingCarGame:

    def __init__(self, window, base, pipes, bird):
        self.window = window
        self.base = base
        self.pipes = pipes
        self.bird = bird

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.bird = Bird(RESTART_X, RESTART_Y)
                        self.pipes = [Pipe(WIN_WIDTH + 50)]
                        self.base = Base(BASE)
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_SPACE:
                        self.bird.jump()

            if not game_over:
                self.bird.move()
                for pipe in self.pipes:
                    pipe.move()
                    if pipe.collide(self.bird):
                        game_over = True
                    if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                        self.pipes.remove(pipe)
                    if not pipe.passed and pipe.x < self.bird.x:
                        pipe.passed = True
                        self.score += 1
                        self.pipes.append(Pipe(WIN_WIDTH + 50))
                self.base.move()

                if self.bird.y + self.bird.img.get_height() >= BASE or self.bird.y < 0:
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display()
        pygame.quit()

    def display(self):
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (255, 255, 255))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [7]:
def main():
    bird = Bird(RESTART_X, RESTART_Y)
    pipes = [Pipe(300)]
    base = Base(BASE)
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = FlappyBirdGame(window, base, pipes, bird)
    game.gameplay()

main()

### Model Creation and Training

In [35]:
"""
AI Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 150
RESTART_Y = 300
BASE = 650

class FlappyBirdGameAI:

    def __init__(self, window, base, pipes, birds, genes, nets):
        self.window = window
        self.base = base
        self.pipes = pipes

        self.birds = birds
        self.genes = genes
        self.nets = nets

        self.clock = pygame.time.Clock()
        self.running = True
        self.gen = 0
        self.score = 0

    def gameplay(self):
        add_pipe = False
        while self.running:
            self.clock.tick(100)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.K_ESCAPE:
                    self.running = False
                    raise neat.CompleteGenome(self.birds[0], self.genes[0], self.nets[0])

            pipe_ind = 0
            if len(self.birds) > 0:
                if len(self.pipes) > 1 and self.birds[0].x > self.pipes[0].x + self.pipes[0].PIPE_TOP.get_width():
                    pipe_ind = 1
            else:
                self.running = False
                break

            for x, bird in enumerate(self.birds):
                self.genes[x].fitness += 0.1
                bird.move()
                output = self.nets[x].activate((bird.y, abs(bird.y - self.pipes[pipe_ind].height), abs(bird.y - self.pipes[pipe_ind].bottom)))
                if output[0] > 0.5:
                    bird.jump()

            for pipe in self.pipes:
                pipe.move()
                for i, bird in enumerate(self.birds):
                    if pipe.collide(bird):
                        self.genes[i].fitness -= 1
                        self.birds.pop(i)
                        self.nets.pop(i)
                        self.genes.pop(i)

                    if not pipe.passed and pipe.x < bird.x:
                        pipe.passed = True
                        add_pipe = True
                        
                if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                    self.pipes.remove(pipe)

            for i, bird in enumerate(self.birds):
                if bird.y + bird.img.get_height() >= BASE or bird.y < 0:
                    self.birds.pop(i)
                    self.nets.pop(i)
                    self.genes.pop(i)

            if add_pipe:
                self.score += 1
                for gene in self.genes:
                    gene.fitness += 5
                pipe = Pipe(WIN_WIDTH, GAP = 200 - 1.5*self.score, VEL = 5 + 0.05*self.score)
                self.base.VEL = pipe.VEL
                self.pipes.append(pipe)
                add_pipe = False

            self.base.move()

            self.display(DRAW_LINES = True)
        pygame.quit()

    def display(self, DRAW_LINES = False):
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        for bird in self.birds:
            bird.draw(self.window)

        if DRAW_LINES and len(self.birds) > 0:
            pipe_ind = 0
            if len(self.pipes) > 1 and self.birds[0].x > self.pipes[0].x + self.pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
            for bird in self.birds:
                pygame.draw.line(self.window, (255, 0, 0), (bird.x + bird.img.get_width()//2, bird.y + bird.img.get_height()//2), (self.pipes[pipe_ind].x + self.pipes[pipe_ind].PIPE_TOP.get_width()//2, self.pipes[pipe_ind].height), 5)
                pygame.draw.line(self.window, (255, 0, 0), (bird.x + bird.img.get_width()//2, bird.y + bird.img.get_height()//2), (self.pipes[pipe_ind].x + self.pipes[pipe_ind].PIPE_BOTTOM.get_width()//2, self.pipes[pipe_ind].bottom), 5)
        
        text = pygame.font.SysFont("norwester", 25).render(f"Generation: {self.gen}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 40))
        text = pygame.font.SysFont("norwester", 25).render(f"Alive: {len(self.birds)}", 1, (255, 255, 255))
        self.window.blit(text, (10, 70))

        pygame.display.update()

In [39]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
generation = 0

def main(genomes, config):
    global generation

    birds = []
    genes = []
    nets = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(RESTART_X, RESTART_Y))
        g.fitness = 0
        genes.append(g)

    pipes = [Pipe(300)]
    base = Base(BASE)
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    try:
        pygame.font.init()
        game = FlappyBirdGameAI(window, base, pipes, birds, genes, nets)
        game.gen = generation
        game.gameplay()
    except neat.CompleteGenome as e:
        raise e

    generation += 1

try:
    winner = population.run(main, 50)
except neat.CompleteGenome as e:
    winner = e.genome

with open("Models/FlappyBirdAI.pkl", "wb") as genome_file:
    pickle.dump(winner, genome_file)
print(f"The Best Genome:\n{winner}")


 ****** Running generation 0 ****** 

Population's average fitness: 3.45000 stdev: 9.19312
Best fitness: 43.50000 - size: (1, 3) - species 1 - id 16
Average adjusted fitness: 0.058
Mean genetic distance 1.685, standard deviation 0.582
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     43.5    0.058     0
Total extinctions: 0
Generation time: 3.573 sec

 ****** Running generation 1 ****** 

Population's average fitness: 3.06000 stdev: 4.76187
Best fitness: 21.70000 - size: (1, 3) - species 1 - id 16
Average adjusted fitness: 0.095
Mean genetic distance 1.878, standard deviation 0.534
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     21.7    0.095     1
Total extinctions: 0
Generation time: 1.904 sec (2.739 average)

 ****** Running generation 2 ****** 

Population's average fitness: 92.97000 stdev: 1

### Genome Loading

In [40]:
with open("FlappyBirdAI.pkl", "rb") as genome_file:
    genome = pickle.load(genome_file)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

model = neat.nn.FeedForwardNetwork.create(genome, config)

In [45]:
print(model.activate((30, 100, 100)))

[-1.0]
